<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T045416Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D11734d8123f8cf4f816865be109ed6a0a1a15661ba0027cb882acf2741bb7416923782df6d680c99fcf0ff7c96ab1b05708dd8b3a127c4528ed7f5caa84414bce3f8d548c148781d99ee579d706c91cfc56ad597b1a5aa0733c1e87404ac5f999fa29d5e26783e81b4bae3ad00b533533411a363f6ef8103d744066f3bf29f5faaa0ff9fa938ae94e71f8207d303c9ff587ffa6f6eb8f8349fc39d570d2970348aed3a2867bc22d17791332fe724b83eaca84f7a38c8f45ff27c700ff1f3baa4eb441625283f2b2b7d160af494b84e87c412a47de292babb6a5abac9e2d3998feddcabb040ce5111f77480f0e43467ae0d2b97e55b2d6772fb6146f4ce816024,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T045416Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D026905eca9c2f346ae156a3e908bf2e86349e2a9500786066cd9bb0ec0d7c9f6be9c0458b6cb5ebf5bca2f6fcc08ea82c19b3554c4e90b9cb8abebad5a51a572cd7143d7c5fef2d13dd7097062e23b27045488f6ec0050e4325f6355f77547da2d97158855eb6bbcb488eee9c8ad688380a570df0e2abb53e92092afd506e678320b596f4e4eac890615591ed81192feef991fcdac7b3996eae5d8b7755f4f80f7e91075ca3e8d8cb42d9d1d75f5f8854a18042ade5e8919d0520ce9e7d111c9077445404531f4e7e574dcb2cf58d1402ddc56822611a74248d55fbf8ae86883a4990f1d92626aa3b35a80d45e72229b2c7cabfc85b7d6b995093d1891ff7978,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T045416Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2be412eeb95aa0b520cdf8a956ae6c46d4968ac128c7bec7115e19a75ef3a9c19c70fa0d1d351880f9307c62880fca43fb626f4dd82cf90b3249ac63be89475f1219164786bbfa36a5a2de7469a61862ae68b55b322711a10bd2e6b242eb4a9b9b96db4e402013bd8212c6b7b6cb2eb6246fde9bcfd7e0f12692cdc7bcfd9ded9e5620bbc43a2a3af990d7a43f5f167e13791aae37d441f4f49d4724609f3f6db5f4911c3d8bfc41a055a46a319fcf752bef3b54306801faeea69babcd94eeb4f4e6838c64e1377247a1de84e2468398e9ebda5bdda543864f6f78c99341c94947979372fc61905b0b795af18235e28342dcf490e863bb73ed25d3747bf139ae'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.postimg.cc/Nfpn7mxR/gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) with a few-shot strategy using some questions of [Stackoverflow python question kaggle dataset](https://www.kaggle.com/datasets/stackoverflow/pythonquestions) and updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) supported by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models and less code changes on the architecture when you decide to test other providers

## Requirements

In [ ]:
!pip install chromadb #!pip install chromadb==0.3.26
# !pip install ydata-profiling #!pip install ydata-profiling==4.6.1
!pip install langchain #!pip install langchain==0.0.345 ##!pip install langchain-core==0.1.31
#!pip install pydantic  #!pip install pydantic==1.10.14
!pip install sentence-transformers   #!pip install sentence-transformers==2.6.1
!pip install InstructorEmbedding  #!pip install InstructorEmbedding==1.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB

In [ ]:
#base
import os
import json
import re
import uuid
import time
import csv
import pandas as pd
import numpy as np
from typing import *
from IPython.display import display, Markdown
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Classes

I use object-oriented Python programming 💻 to develop the chatbot, using the classes 📁 in the corresponding order to support the Agent class that is the final service of the chatbot 🤖.

### Questions and answers data



This class fit the data of [Stackoverflow python question kaggle dataset](https://www.kaggle.com/datasets/stackoverflow/pythonquestions) 📚 to use it in a fine-tuned format ⚙️ or a list of dicts 🗂️... it depends on the case of use you need! it also will be use to select some samples to make the few-shoot strategy

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        df_qa = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        df_qa = df_qa[df_qa['answer_score'] > 5].copy()
        df_qa = df_qa[df_qa['Tag']=='python'].copy()
        df_qa['Answer'] = df_qa['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        return df_qa

    def get_fine_tunning_data(self):
        """Returns a list of formatted user-assistant strings."""
        df_qa_data = self.qa_data()
        data = [
            f"<-change-of-interlocutor->user:\n{row['Question']}\n<-change-of-interlocutor->assistant:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data
    def get_qa_data(self):
        """Returns a list of records dictionaries """
        df_qa_data = self.qa_data()
        data = df_qa_data[
            [
                'Question',
                'Answer'
            ]
        ].to_dict(orient='records')
        return data


### Python Enhancement Proposals data

The class Peps scrape all the oficial [pep](https://peps.python.org/) 🔎 to get all the best practices of this programming language 🐍 and their latest features ✨, disposed to uses in the RAG system for the chatbot

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text.replace('"""',"'") for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

The embeder class is the method that lets you embed the data given as a list of strings ➡️📦. It will support the retriever for the RAG system 🔍

In [ ]:
class Embeder:
    """
    Creates text document embeddings (numerical representations) for semantic search,
    similarity comparison, and related natural language processing tasks. Offers the choice
    between a larger model for richer embeddings or a smaller, more efficient model.

    Attributes:
        large (bool): Controls the size of the embedding model. When set to True, uses a
                      larger model for richer embeddings (default: True).
        model (str): The name of the embedding model to use. Changes based on the value of 'large'.
        device (str): The device to use for computation. Options include "cpu" or, if available, "cuda"
                      for GPU acceleration (default: "cuda").
    """

    def __init__(self, large: bool = False):
        """
        Initializes the Embeder instance with preferences for model size.

        Args:
            large (bool, optional): If True, initializes with the larger embedding model.
                                    Defaults to False for the smaller, faster model.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device = "cuda"  # If GPU is available
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        """
        Configures the embedding pipeline based on the selected model size.

        Returns:
           An embedding object from either the HuggingFaceInstructEmbeddings class
           (for the large model) or the SentenceTransformerEmbeddings class (for the smaller model).
        """
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(model_name=self.model)
        return embed

    def run(self, docs_list: list):
        """
        Generates embeddings for a given list of text documents.

        Args:
            docs_list (list): A list of text documents to embed.

        Returns:
            List[List[float]]: A list of lists, where each inner list represents the
                               numerical embedding for a corresponding document in the input.
        """
        if docs_list is None:
            docs_list = ['']  # Prevent errors with empty input

        embed = self.instructor()  # Get the appropriate embedding object
        return embed.embed_documents(docs_list)


### Prompt

As a good practice, I consider the prompt as the instruction you use to get what you need from an LLM 🧠. It includes all the parameters the LLM needs in every attempt.  Any change in a parameter will generate a different output 🔄.  Therefore, the input is the whole prompt instruction plus all its parameters.

Additionally, it's defined in an external object (like a NoSQL database 🗄️). This allows for easy updates outside of the code, making changes simpler for non-technical people 👍.

In [ ]:
prompt_config = [{
    "prompt_id": "0.0.1",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 1,
    "max_length": 250,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.2",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 1,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.3",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than once the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.4",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures in no more than 1000 characters
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than once the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}]

### Settings

This class aims to load the project's environment variables 🔐 (in this case, Kaggle secrets). Additionally, it disposes of the prompt, as previously defined, to use it according to the team's chosen version 🎯 and for the method that needs those secrets to work ✨.

In [ ]:
class Settings:
    """
    Manages and loads external configuration secrets for the application.  Utilizes a UserSecretsClient
    to securely retrieve sensitive configuration values.
    """

    def __init__(self):
        """
        Initializes the Settings class and retrieves configuration secrets.
        """
        self.user_secrets = UserSecretsClient()  # Create an instance for accessing secrets

        # Load individual secrets (add descriptions for clarity)
        self.CHUNK_SIZE = self.user_secrets.get_secret("CHUNK_SIZE")  # The size of data chunks for processing
        self.CHUNK_OVERLAP = self.user_secrets.get_secret("CHUNK_OVERLAP")  # Ooverlap between data chunks
        self.CHROMA_NAME_INDEX = self.user_secrets.get_secret("CHROMA_NAME_INDEX")  # Vectorial Database identifier
        self.MAX_MEMORY = self.user_secrets.get_secret("MAX_MEMORY")  # Max memory limit to fit the max_lenght paramether
        self.K = self.user_secrets.get_secret("K")  # Could be a parameter for an algorithm
        self.NN_THRESHOLD = self.user_secrets.get_secret("NN_THRESHOLD")  # A threshold for a neural network or similarity metric
        self.PROMPT_ID = self.user_secrets.get_secret("PROMPT_ID")  # An identifier for a prompt (likely in a text-based task and configs)

        # Given that 'prompt_config' is teh previous source of prompt definitions:
        self.prompt = [prompt for prompt in prompt_config if prompt['prompt_id'] == self.PROMPT_ID][0]


### Retriever

The retriever is the engine of the RAG system ⚙️. It uses config settings stored on the Settings class 📁 and the Embeder class to populate the Chroma vector database 🚀. This setup needs to be done once and then can be loaded persistently.

* when setting up the vectorial database, This process may take a few minutes the first time ⏳, so please be PATIENT!

In [ ]:
embeder = Embeder()
settings = Settings()
peps = Peps()
class Retriever:
    """
    Retrieves relevant text chunks from a Chroma vectorial database based on a query. Leverages external
    configuration settings (Settings) and an embedding model (Embeder).

    Handles text splitting, vector database loading, and querying.
    """

    def __init__(self):
        """
        Initializes the Retriever class. Configures the text splitter using settings and stores the
        index name for the Chroma database. Initializes the Chroma vectorial_db object.
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.data = peps.jsonl_format()
        self.documents = self.text_splitter.split_documents(self._prepare_documents())

        # Initialize the Chroma vectorial database with preprocessed data
        self.vectorial_db = Chroma.from_documents(
            documents=self.documents,
            embedding=embeder.instructor()
        )

    def _prepare_documents(self):
        """
        Loads and preprocesses data for storage in the Chroma database.
        """
        documents = []
        for obj in self.data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
        ):
        """
        Retrieves the most similar text chunks from the Chroma database based on a given query.
        """
        # Perform the similarity search
        try:
            res = self.vectorial_db.similarity_search_with_score(message, k=k)
            # Filter and return results
            relevant_results = list(
                set(  # Remove duplicates
                    [
                        vector[0].page_content
                        for vector in res
                        if vector[1] < threshold  # Apply similarity threshold
                    ]
                )
            )

            return [
                result.replace('"""',"'") # Clean up possible docString retrieved from documented code
                for result in relevant_results
            ]
        except Exception as e:
            return []



### Message

The message class lets you store every system, user, and assistant message in an organized structure 💬. This includes unique IDs and timestamps for each message generated during the conversation.

It uses a local JSON file 📄 to store chat history (including few-shot samples to guide the model). In a real-world scenario, this file would represent a NoSQL database 🗄️  capable of handling chats from many users when the chatbot is deployed as a large-scale web service 🌐.

It use the Setting class to meet the main propmpt configs in the run time 😃 🏃‍♀️  

In [ ]:
class Message:
    """
    Represents a message within a conversational context. Stores information about the message's role,
    content, timestamp, and manages saving and loading conversation history.
    """

    def __init__(
        self,
        role: str,
        content: str,
        timestamp: str = int(time.time()),
        prompt_id: str = settings.PROMPT_ID
    ):
        """
        Initializes a Message object.

        Args:
            role (str): Indicates the role of the sender (e.g., 'user', 'system').
            content (str): The text content of the message.
            timestamp (str, optional): A timestamp for the message (defaults to the current time).
            prompt_id (str, optional): Identifier relating to a specific prompt configuration (from settings).
        """

        self.reply_id = str(uuid.uuid4())  # Generate a unique ID for the message
        self.role = role
        self.content = content
        self.timestamp = timestamp
        self.file = 'data/history.json'  # File for storing conversation history

        # Ensure the history file exists
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if it doesn't exist

    def reply(self):
        """
        Formats a basic reply message structure.

        Returns:
            dict: A dictionary containing the message's reply ID, role, content, and timestamp.
        """
        return {
            'reply_id': self.reply_id,
            'role': self.role,
            'content': self.content,
            'timestamp': self.timestamp
        }

    def system_reply(self):
        """
        Generates a system reply using the prompt configuration.

        Returns:
            dict: A dictionary containing the system reply, including ID, role, timestamp,
                  and text content derived from settings.
        """
        prompt = settings.prompt
        return {
            'reply_id': self.reply_id,
            'role': 'system',
            'content': prompt['system'],
            'timestamp': self.timestamp
        }

    def new_chat(self):
        """
        Starts a new chat by initializing the history file.

        Returns:
            list: A list with the initial system reply and the user's message.
        """
        init_chat = [self.system_reply(), self.reply()]
        json.dump(init_chat, open(self.file, 'w'))
        return init_chat

    def update(self):
        """
        Updates the conversation history file by appending the current reply.
        """
        dict_history = json.load(open(self.file))
        dict_history.append(self.reply())
        json.dump(dict_history, open(self.file, 'w'))

    def restart_history(self):
        """
        Clears the conversation history file.
        """
        json.dump([], open(self.file, 'w'))


### Model Gemma

This is the brain of the agent! 🧠 The Gemma class lets you load the model 💪 and tokenizer ⚙️ using those smart settings from the Setting class. Then, you can chat with the model using the powerful GPU device 🚀.

In [ ]:
class Gemma:
    """
    Implements a conversational AI chatbot powered by Gemma large language model. Initializes the model,
    tokenizer, and prepares settings from a configuration.
    """

    def __init__(self):
        """
        Initializes the Gemma chatbot instance.

        Loads the language model and its corresponding tokenizer from the settings configuration.
        Prepares the model for use on a GPU (if available).
        """
        self.prompt = settings.prompt  # Load prompt settings

        # Load language model and tokenizer
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16  # Use half-precision for efficiency (if supported)
        ).cuda()  # Move model to GPU (if available)

    def chat(self, context: str):
        """
        Generates a chatbot response based on the provided conversational context.

        Args:
            context (str): The conversational input text.

        Returns:
            str: The generated text response from the chatbot.
        """
        # Prepare input for the language model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {
            k: v.to("cuda") for k, v in input_ids.items()  # Move tensors to GPU
        }

        # Generate a response with the language model
        outputs = self.gemma.generate(
            **input_ids,
            max_length=self.prompt['max_length']  # Control response length
        )

        # Decode and return the generated text
        return self.tokenizer.decode(outputs[0])


In [ ]:
class GemmaSubtle:
    """
    Implements a conversational AI chatbot powered by Gemma large language model. Initializes the model,
    tokenizer, and prepares settings from a configuration.
    """

    def __init__(self):
        """
        Initializes the Gemma chatbot instance.

        Loads the language model and its corresponding tokenizer from the settings configuration.
        Prepares the model for use on a GPU (if available).
        """
        self.prompt = settings.prompt  # Load prompt settings

        # Load language model and tokenizer
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16  # Use half-precision for efficiency (if supported)
        ).cuda()  # Move model to GPU (if available)

    def chat(self, context: str):
        """
        Generates a chatbot response based on the provided conversational context.

        Args:
            context (str): The conversational input text.

        Returns:
            str: The generated text response from the chatbot.
        """
        # Prepare input for the language model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {
            k: v.to("cuda") for k, v in input_ids.items()  # Move tensors to GPU
        }

        # Generate a response with the language model
        outputs = self.gemma.generate(
            **input_ids,
            max_length=self.prompt['max_length'],  # Control response length
            temperature=1
        )

        # Decode and return the generated text
        return self.tokenizer.decode(outputs[0])

In [ ]:
gemmasubtle = GemmaSubtle()
gemmasubtle.chat('who are you?')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Expert Python Agent/Assistant

Finally, the agent class brings it all together! It uses the retriever 🔍 to implement the RAG system and the Gemma class 🧠 as the powerful brain. Together, they answer questions and the agent performs some final preprocessing ✨ to deliver the polished answer

In [ ]:
%%time
retriever = Retriever()
gemma = Gemma()

class Agent:
    """
    Implements a conversational AI agent that leverages a knowledge database for information retrieval
    and integrates with a generative language model (Gemma) for response generation. Manages conversation
    history and question-answering logic.
    """

    def __init__(self):
        """
        Initializes the Agent, loading prompt settings and ensuring the conversation history file exists.
        """
        self.prompt = settings.prompt
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if necessary

        self.token = '\n<-change-of-interlocutor->'  # Token to separate speakers in the chat history
        self.memory_lenght = int(settings.MAX_MEMORY)

    def augmented_question(self, question: str):
        """
        Enhances the user's question with relevant technical documentation.

        Args:
            question (str): The user's original question.

        Returns:
            str: The question augmented with technical documentation (if found), otherwise the original question.
        """
        tech_docs = retriever.query(question)
        if len(tech_docs) > 0:
            docs = self.prompt['technical_documentation']
            tech_docs = '\n* '.join(tech_docs)
            docs = docs.format(technical_documentation=tech_docs)
            augmented_reply = f"""{question}{docs}"""
            return augmented_reply
        else:
            return question

    def memory(self, question: str):
        """
        Prepares the conversational context (memory) for the language model.

        Args:
            question (str): The user's current question.

        Returns:
            str: Formatted conversation history with a clear separation between speakers, ready
                 for input to the language model.
        """

        dict_history = json.load(open(self.file))
        message = Message(
            role='user',
            content=self.augmented_question(question)
        )

        if len(dict_history) > 0:
            message.update()  # Add the latest message to history
            full_chat = json.load(open(self.file))
        else:
            full_chat = message.new_chat()  # Start a new conversation

        if len(full_chat) > self.memory_lenght:
            full_chat = [full_chat[0]] + full_chat[-self.memory_lenght:] #limiting the few-shot prompt to fit max_lenght

        return '\n'.join(
            [
                self.token + reply['role'] + ': ' + reply['content']
                for reply in full_chat
            ]
        ) + self.token + 'assistant:'

    def get_answer(self, full_response: str, question: str):
        """
        Extracts the relevant answer from the language model's generated response.

        Args:
            full_response (str): The complete response generated by the language model.
            question (str): The user's original question.

        Returns:
            str: The extracted answer.
        """

        answer_list = full_response.split(self.token)
        pos_list = [
            pos
            for pos, answer
            in enumerate(answer_list)
            if question in answer
        ]
        tokened_answer = answer_list[pos_list[0] + 1]
        answer = tokened_answer.split('assistant:')[1]
        message = Message(
            role='assistant',
            content=answer
        )
        message.update()  # Update conversation history
        return answer

    def chat(self, question: str, verbose: bool = False):
        """
        Manages the core interaction with the Agent.

        Args:
            question (str): The user's query.
            verbose (bool, optional): If True, prints the language model's full response (default: False).

        Returns:
            str: The Agent's answer to the user's question.
        """

        memory = self.memory(question)  # Build conversational context
        full_response = gemma.chat(context=memory)  # Generate response
        if verbose:
            print(full_response)
        answer = self.get_answer(full_response, question)  # Extract the answer
        return display(Markdown(answer))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 12min 8s, sys: 14.1 s, total: 12min 22s
Wall time: 11min 33s


# Data

## fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_qa = PythonQAData.get_qa_data()
print(python_qa[1486])

{'Question': 'How can I split a file in python?', 'Answer': "This one splits a file up by newlines and writes it back out. You can change the delimiter easily. This can also handle uneven amounts as well, if you don't have a multiple of splitLen lines (20 in this example) in your input file.\n\nsplitLen = 20         # 20 lines per file\noutputBase = 'output' # output.1.txt, output.2.txt, etc.\n\n# This is shorthand and not friendly with memory\n# on very large files (Sean Cavanagh), but it works.\ninput = open('input.txt', 'r').read().split('\\n')\n\nat = 1\nfor lines in range(0, len(input), splitLen):\n    # First, get the list slice\n    outputData = input[lines:lines+splitLen]\n\n    # Now open the output file, join the new slice with newlines\n    # and write it out. Then close the file.\n    output = open(outputBase + str(at) + '.txt', 'w')\n    output.write('\\n'.join(outputData))\n    output.close()\n\n    # Increment the counter\n    at += 1\n\n"}
CPU times: user 16.2 s, sys: 9

In [ ]:
print(
'\n* Mean lenght of a python common question:',
sum([len(q['Answer']) for q in python_qa])/len(python_qa),
'\n* Standard deviation of length a common Python question:',
np.std([len(q['Answer']) for q in python_qa])
)


* Mean lenght of a python common question: 973.8736229099751 
* Standard deviation of length a common Python question: 1200.4630038643415


In [ ]:
print(
'\n* Mean lenght of a python common answer:',
sum([len(q['Question']) for q in python_qa])/len(python_qa),
'\n* Standard deviation of length a common Python question:',
np.std([len(q['Question']) for q in python_qa])
)

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_fine_tunning_data()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][:1000]
)

Python Questions loaded: 33313 
 
Sample question-answer:
 <-change-of-interlocutor->user:
Why can yield be indexed?
<-change-of-interlocutor->assistant:
You are not indexing. You are yielding a list; the expression yield[0] is really just the same as the following (but without a variable):

lst = [0]
yield lst


If you look at what next() returned you'd have gotten that list:

&gt;&gt;&gt; def gen1():
...   t = yield[0]
...   assert t
...   yield False
...
&gt;&gt;&gt; g = gen1()
&gt;&gt;&gt; next(g)
[0]


You don't have to have a space between yield and the [0], that's all.

The exception is caused by you trying to apply the subscription to the contained 0 integer:

&gt;&gt;&gt; [0]        # list with one element, the int value 0
[0]
&gt;&gt;&gt; [0][0]     # indexing the first element, so 0
0
&gt;&gt;&gt; [0][0][0]  # trying to index the 0
Traceback (most recent call last):
  File "&lt;stdin&gt;", line 1, in &lt;module&gt;
TypeError: 'int' object is not subscriptable


If you want t

## few-shot data

In [ ]:
few_shot_list=[7,6009,12,36,8034,130,141,537,1057,5042]
for i, q in enumerate(few_shot_list):
    print('_'*100,f'\n{i}.',python_qa[q])

____________________________________________________________________________________________________ 
0. {'Question': "How do I use Python's itertools.groupby()?", 'Answer': 'As Sebastjan said, you first have to sort your data. This is important.\n\nThe part I didn\'t get is that in the example construction\n\ngroups = []\nuniquekeys = []\nfor k, g in groupby(data, keyfunc):\n   groups.append(list(g))    # Store group iterator as a list\n   uniquekeys.append(k)\n\n\nk is the current grouping key, and g is an iterator that you can use to iterate over the group defined by that grouping key. In other words, the groupby iterator itself returns iterators.\n\nHere\'s an example of that, using clearer variable names:\n\nfrom itertools import groupby\n\nthings = [("animal", "bear"), ("animal", "duck"), ("plant", "cactus"), ("vehicle", "speed boat"), ("vehicle", "school bus")]\n\nfor key, group in groupby(things, lambda x: x[0]):\n    for thing in group:\n        print "A %s is a %s." % (thing[

## RAG data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

corpus lenght: 645 
section <n> lenght: 40928 
sample text:

 implemented ASAP, and the handling of regular iterators, which is a
larger but more relaxed project that can’t start until 3.7 at the
earliest. But since the changes are closely related, and we probably
don’t want to end up with async iterators and regular iterators
diverging in the long run, it seems useful to look at them together.


Background and motivation
Python iterables often hold resources which require cleanup. For
example: file objects need to be closed; the WSGI spec adds a close method
on top of the regular iterator protocol and demands that consumers
call it at the appropriate time (though forgetting to do so is a
frequent source of bugs);
and PEP 342 (based on PEP 325) e
CPU times: user 48.8 s, sys: 665 ms, total: 49.4 s
Wall time: 2min 1s


# RAG

### Testing the embeder class

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["Beautiful is better than ugly."])[0]
print(len(vec))
vec[:10]

384
CPU times: user 1.45 s, sys: 272 ms, total: 1.72 s
Wall time: 2.27 s


[0.19154123961925507,
 0.20040714740753174,
 0.29264503717422485,
 0.7032436728477478,
 0.1398404985666275,
 -0.16274164617061615,
 -0.18401199579238892,
 -0.35343530774116516,
 0.08375284075737,
 0.1366858184337616]

### Testing the Retriver

In [ ]:
%%time
question = """
Develop a Python code snippet to print the diamond structure with the specified number of rows.
The program should follow the Fibonacci sequence for the number of characters per row
and validate input to ensure it is an odd number.
"""
retriever.query(
    question,
    k=100,
    threshold=15
)

CPU times: user 25.6 ms, sys: 37 µs, total: 25.6 ms
Wall time: 24.8 ms


['Adding an optimization to the Python compiler and VM\nwhich detects the above if-elif-else construct and\ngenerates special opcodes for it which use a read-only\ndictionary for storing jump offsets.\nAdding new syntax to Python which mimics the C style',
 'In Python, we simply use isinstance() together with the __match_args__\nfield of a class to check whether an object has the correct structure and\nthen transform some of its attributes into a tuple.  For the Node example',
 'They call either the Python class hooks (if either of the objects is a Python\nclass instance) or the C type’s number or sequence methods.\nThe new bytecodes are:\nINPLACE_ADD\nINPLACE_SUBTRACT\nINPLACE_MULTIPLY\nINPLACE_DIVIDE\nINPLACE_REMAINDER',
 'allowed as a literal decimal in Python 3.0.  If this is the\nright thing to do, this can easily be covered in an additional\nPEP.  This proposal only takes the first step of making ‘0123’',
 'int PyInitConfig_SetWStrList(PyInitConfig *config, const char *name, size

In [ ]:
%%time
question = """
PyStatus PyConfig_SetBytesString(PyConfig *config, wchar_t * const *config_str, const char *str):\nDecode str using Py_DecodeLocale() and set the result into\n*config_str.\nPreinitialize Python if needed.
"""
retriever.query(question,
    k=100,
    threshold=15
)

CPU times: user 22.6 ms, sys: 992 µs, total: 23.6 ms
Wall time: 22.9 ms


['Py_INCREF(), Py_DECREF()\nPy_XINCREF(), Py_XDECREF()\nPyObject_INIT(), PyObject_INIT_VAR()\n_PyObject_GC_TRACK(), _PyObject_GC_UNTRACK(), _Py_Dealloc()\n\nMacros converted to regular functions in Python 3.9:',
 'PyStatus PyConfig_SetString(PyConfig *config, wchar_t * const *config_str, const wchar_t *str):\nCopy the wide character string str into *config_str.\nPreinitialize Python if needed.',
 'decode the strings in Py_InitializeFromInitConfig().\nThe problem is that users asked for PyInitConfig_Get*() functions.\nFor example, PyInitConfig_GetStr() must decode the string from the\nlocale encoding and then encode it to the UTF-8 encoding.',
 'PyConfig_Clear(config)\nPyConfig_InitIsolatedConfig()\nPyConfig_InitPythonConfig()\nPyConfig_Read(config)\nPyConfig_SetArgv(config, argc, argv)\nPyConfig_SetBytesArgv(config, argc, argv)\nPyConfig_SetBytesString(config, config_str, str)',
 'Trying to set these options after Python preinitialization fails with an\nerror.\nPyInitConfig_SetStrLocal

In [ ]:
%%time
question = """
what PEPs Introduces the concept of decorators, a powerful feature in Python.
"""
retriever.query(question,
    k=100,
    threshold=15
)

CPU times: user 22.5 ms, sys: 990 µs, total: 23.5 ms
Wall time: 22.6 ms


['The PEPs category of the Python Discourse\nis the preferred choice for most new PEPs,\nwhereas historically the Python-Dev mailing list was commonly used.\nSome specialized topics have specific venues, such as',
 'documenting the design decisions that have gone into Python.  The PEP\nauthor is responsible for building consensus within the community and\ndocumenting dissenting opinions.\nBecause the PEPs are maintained as text files in a versioned',
 'Introduction\nThis PEP describes the “range literal” proposal for Python 2.0.\nThis PEP tracks the status and ownership of this feature, slated\nfor introduction in Python 2.0.  It contains a description of the',
 'the Typing category and the Packaging category on the Python\nDiscourse for typing and packaging PEPs, respectively.\nIf the PEP authors are unsure of the best venue,\nthe PEP Sponsor and PEP editors can advise them accordingly.',
 'Introduction\nThis PEP describes the “attribute docstring” proposal for Python\n2.0.  This PEP 

In [ ]:
%%time
question = """
what is python??
"""
retriever.query(question)

CPU times: user 20.4 ms, sys: 1.99 ms, total: 22.4 ms
Wall time: 20.8 ms


[]

# Chatbot

## Loading few-shot to the memory

In [ ]:
file = 'data/history.json'
message = Message(
    role='sistem',
    content='foo'
)
init_chat = [message.system_reply()]
json.dump(init_chat, open(file, 'w'))
for i, q in enumerate(few_shot_list):
    sample = python_qa[q]
    mesage = Message(
        role='user',
        content=sample['Question']
    )
    reply = mesage.update()
    mesage = Message(
        role='assitant',
        content=sample['Answer']
    )
    reply = mesage.update()

history = json.load(open(file))
history

[{'reply_id': '8bcef358-5e5c-430a-a4a5-f7620aacb718',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n6. DO NOT repeat more than once the same sentence!\n\nTAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION\n\nFor each question you answer perfectly, I will pay you USD

### Testing the Agent system memory

In [ ]:
agent = Agent()
print(agent.memory("""
what PEPs Introduces the concept of decorators, a powerful feature in Python.
"""))


<-change-of-interlocutor->system: 
Act as a Python programming language expert assistant 
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary 
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than once the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct ans

# Gemma Python Chat

In [ ]:
%%time
agent = Agent()
question = 'what is python??'
print(agent.chat(question))

 Python is a high-level, interpreted, object-oriented programming language. It is often referred to as a "high-level" language because it is designed to be easy to use for programmers while offering the powerful features of low-level languages. Python is an interpreted language, which means that the source code is executed line by line by the Python interpreter. Python is an object-oriented language, which means that it uses classes and objects to organize and manipulate data. Python is a general-purpose language, which means that it can be used for a wide range of tasks, from simple scripting to complex scientific and engineering applications. Python is a dynamic language, which means that variables can be changed at runtime. Python is a multi-paradigm language, which means that it supports a variety of programming styles, including procedural, object-oriented, and functional. Python is a standard-compliant language, which means that it conforms to the ISO/IEC 9899:1999 standard for the C programming language. Python is a portable language, which means that it can be compiled to run on a wide range of platforms, including Windows, Linux, and Mac OS X. Python is a rich language, which means that it has a large and active community of developers and a rich ecosystem of libraries and tools. Python is a community-driven language, which means that it is developed and maintained by a large and active community of developers. Python is a free and open-source language, which means that it is available for free and can be modified and extended by anyone. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users.


None
CPU times: user 2min 35s, sys: 93.8 ms, total: 2min 35s
Wall time: 2min 35s


In [ ]:
%%time
agent = Agent()
question = 'who are you???'
print(agent.chat(question))

 I am a Python programmer.


None
CPU times: user 58.1 s, sys: 22.8 ms, total: 58.1 s
Wall time: 58 s


In [ ]:
%%time
agent = Agent()
question = 'what is python and how long is it????'
print(agent.chat(question))

  Python is a high-level, interpreted, object-oriented programming language. It is often referred to as a "high-level" language because it is designed to be easy to use for programmers while offering the powerful features of low-level languages. Python is an interpreted language, which means that the source code is executed line by line by the Python interpreter. Python is an object-oriented language, which means that it uses classes and objects to organize and manipulate data. Python is a general-purpose language, which means that it can be used for a wide range of tasks, from simple scripting to complex scientific and engineering applications. Python is a dynamic language, which means that variables can be changed at runtime. Python is a multi-paradigm language, which means that it supports a variety of programming styles, including procedural, object-oriented, and functional. Python is a standard-compliant language, which means that it conforms to the ISO/IEC 9899:1999 standard for the C programming language. Python is a portable language, which means that it can be compiled to run on a wide range of platforms, including Windows, Linux, and Mac OS X. Python is a rich language, which means that it has a large and active community of developers and a rich ecosystem of libraries and tools. Python is a community-driven language, which means that it is developed and maintained by a large and active community of developers and users. Python is a free and open-source language, which means that it is available for free and can be modified and extended by anyone. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used for complex tasks. It is a language that is well-suited for scripting and automation, and it has a large and active community of developers and users. Python is a language that is easy to learn and use, but powerful enough to be used

None
CPU times: user 1min 13s, sys: 28.8 ms, total: 1min 13s
Wall time: 1min 13s


In [ ]:
%%time
agent = Agent()
question = 'my name is juan nice to meet you!!!'
print(agent.chat(question))

  Nice to meet you too, Juan. I'm here to help you with Python programming language questions only, excuse me.

None
CPU times: user 12.3 s, sys: 6.91 ms, total: 12.3 s
Wall time: 12.3 s


In [ ]:
%%time
agent = Agent()
question = 'who are you?!'
print(agent.chat(question))

  I am a Python programmer.


None
CPU times: user 20.2 s, sys: 13.5 ms, total: 20.3 s
Wall time: 20.2 s


In [ ]:
%%time
agent = Agent()
question = 'what is my name??'
print(agent.chat(question))

   Your name is Juan.


None
CPU times: user 20.9 s, sys: 10.7 ms, total: 20.9 s
Wall time: 20.9 s


In [ ]:
%%time
agent = Agent()
question = 'Develop a Python code snippet to print the diamond structure with the specified number of rows. The program should follow the Fibonacci sequence for the number of characters per row and validate input to ensure it is an odd number.'
print(agent.chat(question))

   The program should print the following:

None
CPU times: user 2min 11s, sys: 50.3 ms, total: 2min 11s
Wall time: 2min 11s


In [ ]:
%%time
agent = Agent()
question = 'Create a Python dictionary that stores square roots of numbers from 1 to 15 and cube roots of numbers from 16 to 30, ensuring to keep a precision up to two decimal places.'
print(agent.chat(question))

    The program should print the following:


None
CPU times: user 2min 10s, sys: 47.7 ms, total: 2min 10s
Wall time: 2min 10s


In [ ]:
%%time
agent = Agent()
question = 'give me the python class that do a list comprehension format'
print(agent.chat(question))

    The program should print the following:


None
CPU times: user 2min 48s, sys: 58.6 ms, total: 2min 48s
Wall time: 2min 48s


In [ ]:
%%time
agent = Agent()
question = 'tell me a python joke!'
print(agent.chat(question))

     The program should print the following:



None
CPU times: user 2min 49s, sys: 61.8 ms, total: 2min 49s
Wall time: 2min 49s


In [ ]:
%%time
agent = Agent()
question = 'how to print on hello world?'
print(agent.chat(question))

     The program should print the following:



None
CPU times: user 2min 49s, sys: 68.1 ms, total: 2min 49s
Wall time: 2min 49s


In [ ]:
history = json.load(open(file))
history

[{'reply_id': '8bcef358-5e5c-430a-a4a5-f7620aacb718',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n6. DO NOT repeat more than once the same sentence!\n\nTAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION\n\nFor each question you answer perfectly, I will pay you USD